## ODI CRICKET SCRAPING

In this Project I scraped the match vise scorecard for last 5 years (2018 to 2023). 
The objective of the project was to analyze the the game as well as as based on data recommend best playing 11 for each of the Qualifying Nation.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import re
from alive_progress import alive_bar
import time

In [114]:
# website = "https://www.espncricinfo.com/records/season/team-match-results/2014to15-2014to15?trophy=12"
# result = requests.get(website)
# content = result.text
# soup = BeautifulSoup(content, "html")

#### I scraped the data from Cricinfo, wikipedia as well as other sites.
### steps followed are as following:

1. for each year (2018 - 2023) I scraped all the matched played in that year across all teams,
check the function scrape_match_dataset. 
(which also contained links to the team profile as well as links for the Match detailed scorecard)

2. for each year I went to each match link and scraped both the batting scorecard and bowling scorecard.
(Data scraped in such a way that minimum effort is required in data modelling)
(check the fucntions - scrape_batting_summary, srape_bowling_summary)

3. After this from Match scorecard i also scraped the player links which helped me scrape cruiucla details about each player such as their stats, 
their nationality their preference order etc.

4. Scraped National falgs for each team

5. Because of issue of Lazy loading of player's images had to scrape the pictures using Selenium where I intialized delay time as 20 secounds and it will keep on increasing the time by 5 seconds till 60 seconds, if image is still not loading. 
(code in second notebook along with data modelling and analysis)


### Match_datset

In [2]:
def scrape_match_dataset(website):
    
    website = website
    result = requests.get(website)
    content = result.text
    soup = BeautifulSoup(content, "html")
    table = soup.find('table',class_="ds-w-full")
    
    for row in table.thead.find_all("tr"):
        columns = row.find_all("td")
    column_headers = []
    for i in columns:
        column_headers.append(i.text.strip())
    df_match= pd.DataFrame(columns=column_headers)
    df_match=df_match.rename(columns={"Match Date":"Match_Date","Team 1": "Team_1","Team 2":"Team_2","Scorecard":"Match_id"})
    df_match


    for row in table.tbody.find_all("tr"):
        columns = row.find_all("td")

        if(columns != []):
            #Team_1_link = columns[0].span.a.get("href")
            Team_1 = columns[0].span.a.span.contents[0].strip()
            Team_2 = columns[1].span.a.span.contents[0].strip()
            Winner = columns[2].span.contents[0].strip()
            Margin = columns[3].span.contents[0].strip()
            Ground = columns[4].span.a.span.contents[0].strip()
            Match_Date = columns[5].span.contents[0].strip()
            Scorecard = columns[6].span.a.span.contents[0].strip()
            #print(Scorecard)
        df_match = df_match.append({"Team_1":Team_1,
                                            "Team_2":Team_2,
                                            "Winner":Winner,
                                            "Margin":Margin,
                                            "Ground":Ground,
                                            "Match_Date":Match_Date,
                                            "Match_id":Scorecard},
                                           ignore_index=True)
        
    return df_match


### Match_links

In [3]:
def scrape_match_links(website):
    website = website
    result = requests.get(website)
    content = result.text
    soup = BeautifulSoup(content, "html")
    table = soup.find('table',class_="ds-w-full")
    df_match_links=pd.DataFrame(['Team_1_Link',"Team_2_link","Ground_link","match_link"])
    for row in table.tbody.find_all("tr"):
        columns = row.find_all("td")

        if(columns != []):
            Team_1_link = columns[0].span.a.get("href")
            Team_2_link = columns[1].span.a.get("href")
            Ground_link = columns[4].span.a.get("href")
            Scorecard_link = columns[6].span.a.get("href")

        df_match_links = df_match_links.append({"Team_1_link":Team_1_link,
                                                        "Team_2_link":Team_2_link,
                                                        "Ground_link":Ground_link,
                                                        "match_link":Scorecard_link},
                                                       ignore_index=True)
    df_match_links=df_match_links.drop(columns=0)
    df_match_links = df_match_links.dropna()
    df_match_links=df_match_links.reset_index(drop="True")
    #df_match_links
    return df_match_links

### Batting Scorecard

In [40]:
def scrape_batting_summary(df_match_links,df_match):
    #df match_battingstats()
    df_match_links=df_match_links
    
    
    df_batting_summary = pd.DataFrame(columns=["BATTING","R","B","4s","6s","SR","team","Match_id"])
    df_batting_summary

    for ind in df_match_links.index:
            url ="https://www.espncricinfo.com/"+df_match_links["match_link"][ind]
            result = requests.get(url)
            content = result.text
            soup = BeautifulSoup(content, "html")

            tables = soup.find_all("table")

            bat=0
            for table in tables:
                th_elements = table.find_all('th')
                for th in th_elements:
                    if "BATTING" in th.text:
                        bat+=1

            if bat==2:
                table = tables[0]
                for row in table.tbody.find_all("tr"):
                    columns = row.find_all("td")
                    #print(len(columns))
                    if len(columns) == 8:
                        #if(columns != []):
                        Batsmen = columns[0].a.get("title")
                        Runs = columns[2].strong.contents[0]
                        Balls = columns[3].contents[0].strip()
                        No_4s = columns[5].contents[0].strip()
                        No_6s = columns[6].contents[0].strip()
                        Strike_rate = columns[7].contents[0].strip()
                       #print(1)
                        team = soup.find_all("span",class_="ds-text-title-xs ds-font-bold ds-capitalize")[0].contents[0]
                        Match_id = df_match["Match_id"][ind]
                        df_bat1 =  pd.DataFrame([{"BATTING":Batsmen,
                                                              "R":Runs,
                                                              "B":Balls,
                                                              "4s":No_4s,
                                                              "6s":No_6s,
                                                              "SR":Strike_rate,
                                                              "team":team,
                                                              "Match_id":Match_id}])
                        df_batting_summary = pd.concat([df_batting_summary, df_bat1], ignore_index=True)

                table = tables[2]
                for row in table.tbody.find_all("tr"):
                    columns = row.find_all("td")
                    #print(len(columns))
                    if len(columns) >= 8 :
                        #if(columns != []):
                        Batsmen = columns[0].a.get("title")
                        Runs = columns[2].strong.contents[0]
                        Balls = columns[3].contents[0].strip()
                        No_4s = columns[5].contents[0].strip()
                        No_6s = columns[6].contents[0].strip()
                        Strike_rate = columns[7].contents[0].strip()
                        #print(1)
                        team = soup.find_all("span",class_="ds-text-title-xs ds-font-bold ds-capitalize")[1].contents[0]
                        Match_id = df_match["Match_id"][ind]
            #             df_batting_summary=df_batting_summary.append({"BATTING":Batsmen,
            #                                                   "R":Runs,
            #                                                   "B":Balls,
            #                                                   "4s":No_4s,
            #                                                   "6s":No_6s,
            #                                                   "SR":Strike_rate,
            #                                                   "team":team,
            #                                                   "Match_id":Match_id},
            #                                                  ignore_index=True)
                        df_bat1 =  pd.DataFrame([{"BATTING":Batsmen,
                                                              "R":Runs,
                                                              "B":Balls,
                                                              "4s":No_4s,
                                                              "6s":No_6s,
                                                              "SR":Strike_rate,
                                                              "team":team,
                                                              "Match_id":Match_id}])
                        df_batting_summary = pd.concat([df_batting_summary, df_bat1], ignore_index=True)
    return df_batting_summary

### Bowling_Scorecard

In [41]:
def scrape_bowling_summary(df_match_links,df_match):
    df_bowling_summary = pd.DataFrame(columns=["BOWLING","O","M","R","W","ECON","0s","4s","6s","WD","NB","Match_id","team"])
    df_bowling_summary
    
    df_match_links=df_match_links
    for ind in df_match_links.index:
        url ="https://www.espncricinfo.com/"+df_match_links["match_link"][ind]
        result = requests.get(url)
        content = result.text
        soup = BeautifulSoup(content, "html")

        tables = soup.find_all("table")
        #if len(tables)>=4:
        ball=0
        for table in tables:
            th_elements = table.find_all('th')
            for th in th_elements:
                if "BOWLING" in th.text:
                    ball+=1

        if ball==2:
            table = tables[1]
            for row in table.tbody.find_all("tr",{"class":""}):
                columns = row.find_all("td")
                #print(len(columns[4]))
                Bowlers = columns[0].a.span.contents[0].strip()
                O = columns[1].contents[0].strip()
                M = columns[2].contents[0].strip()
                R = columns[3].contents[0].strip()
                try:
                    W = columns[4].span.strong.contents[0].strip()
                except AttributeError:
                    W = columns[4].strong.contents[0].strip()
                ECON = columns[5].contents[0].strip()
                No_0s = columns[6].contents[0].strip()
                No_4s = columns[7].contents[0].strip()
                No_6s = columns[8].contents[0].strip()
                WD = columns[9].contents[0].strip()
                NB = columns[10].contents[0].strip()
    #             try:
    #                 print(len(soup.find_all("span",class_="ds-text-title-xs ds-font-bold ds-capitalize")))
                team = soup.find_all("span",class_="ds-text-title-xs ds-font-bold ds-capitalize")[1].contents[0]
    #             except:
    #                 import pdb
    #                 pdb.set_trace()
                #print(team)
                Match_id = df_match["Match_id"][ind]
                #print(Bowlers,O,M,R,W,ECON,No_0s,No_4s,No_6s,WD,NB,team,Match_id)
                df1 =  pd.DataFrame([{"BOWLING":Bowlers,
                                "O":O,
                                "M":M,
                                "R":R,
                                "W":W,
                                "ECON":ECON,
                                "0s":No_0s,
                                "4s":No_4s,
                                "6s":No_6s,
                                "WD":WD,
                                "NB":NB,
                                "Match_id":Match_id,
                                "team":team}])

                df_bowling_summary = pd.concat([df_bowling_summary, df1], ignore_index=True)


            table = tables[3]
            for row in table.tbody.find_all("tr",{"class":""}):
                columns = row.find_all("td")
                #print(len(columns[4]))
                Bowlers = columns[0].a.span.contents[0].strip()
                O = columns[1].contents[0].strip()
                M = columns[2].contents[0].strip()
                R = columns[3].contents[0].strip()
                try:
                    W = columns[4].span.strong.contents[0].strip()
                except AttributeError:
                    W = columns[4].strong.contents[0].strip()
                ECON = columns[5].contents[0].strip()

                No_0s = columns[6].contents[0].strip()

                No_4s = columns[7].contents[0].strip()
                No_6s = columns[8].contents[0].strip()
                WD = columns[9].contents[0].strip()
                NB = columns[10].contents[0].strip()
                team = soup.find_all("span",class_="ds-text-title-xs ds-font-bold ds-capitalize")[0].contents[0]
                Match_id = df_match["Match_id"][ind]
                #print(Bowlers,O,M,R,W,ECON,No_0s,No_4s,No_6s,WD,NB,team,Match_id)
                #print(ECON, No_0s,NB)
                df1 =  pd.DataFrame([{"BOWLING":Bowlers,
                                "O":O,
                                "M":M,
                                "R":R,
                                "W":W,
                                "ECON":ECON,
                                "0s":No_0s,
                                "4s":No_4s,
                                "6s":No_6s,
                                "WD":WD,
                                "NB":NB,
                                "Match_id":Match_id,
                                "team":team}])

                df_bowling_summary = pd.concat([df_bowling_summary, df1], ignore_index=True)

    #df_bowling_summary
    return df_bowling_summary

### Player_links

In [30]:
def scrape_player_links(df_match_links):
    df_players = pd.DataFrame(columns=["Player_name","Player_link"])
    
    df_match_links=df_match_links
    for ind in df_match_links.index:
        url ="https://www.espncricinfo.com/"+df_match_links["match_link"][ind]
        result = requests.get(url)
        content = result.text
        soup = BeautifulSoup(content, "html")

        tables = soup.find_all("table")
        ball=0
        for table in tables:
            th_elements = table.find_all('th')
            for th in th_elements:
                if "BOWLING" in th.text:
                    ball+=1

        bat=0
        for table in tables:
            th_elements = table.find_all('th')
            for th in th_elements:
                if "BATTING" in th.text:
                    bat+=1

        if bat==2 & ball==2:

            table = tables[0]
            for row in table.tbody.find_all("tr"):
                columns = row.find_all("td")
                #print(len(columns))
                if len(columns) == 8:
                    #if(columns != []):
                    Batsmen = columns[0].a.get("title")
                    Batsmen_link = columns[0].a.get("href")
                    #print(1)
                    df_players=df_players.append({"Player_name":Batsmen,
                                                  "Player_link":Batsmen_link},
                                                         ignore_index=True)
            table = tables[2]
            for row in table.tbody.find_all("tr"):
                columns = row.find_all("td")
                #print(len(columns))
                if len(columns) == 8:
                    #if(columns != []):
                    Batsmen = columns[0].a.get("title")
                    Batsmen_link = columns[0].a.get("href")
                    #print(1)
                    df_players=df_players.append({"Player_name":Batsmen,
                                                  "Player_link":Batsmen_link},
                                                         ignore_index=True)


            table = tables[1]
            for row in table.tbody.find_all("tr",{"class":""}):
                columns = row.find_all("td")
                #print(len(columns[4]))
                Bowlers = columns[0].a.span.contents[0].strip()
                Bowlers_link = columns[0].a.get("href")
                df_players=df_players.append({"Player_name":Bowlers,
                                                  "Player_link":Bowlers_link},
                                                         ignore_index=True)

            table = tables[3]
            for row in table.tbody.find_all("tr",{"class":""}):
                columns = row.find_all("td")
                #print(len(columns[4]))
                Bowlers = columns[0].a.span.contents[0].strip()
                Bowlers_link = columns[0].a.get("href")
                df_players=df_players.append({"Player_name":Bowlers,
                                                  "Player_link":Bowlers_link},
                                                         ignore_index=True)  
    df_players = df_players.drop_duplicates()
    df_players = df_players.reset_index(drop = True)
    return df_players

### Player_details

In [31]:
def scrape_player_details(df_players):
    #with alive_bar(len(df_players),force_tty=True) as bar:
    df_player_profile = pd.DataFrame(columns = ["Name","Born","Age","Batting_Style","Bowling_Style","Playing_Role",
                                             "National_Team","player_img","career_start","career_latest"])
    #df_player_profile
    
    df_players=df_players
    for ind in df_players.index:
        time.sleep(0.005)
        url ="https://www.espncricinfo.com/"+df_players["Player_link"][ind]
        #print(url)
        result = requests.get(url)
        content = result.text
        soup = BeautifulSoup(content, "lxml")


        #Personal_details
        list_names=["Full Name","Born","Age","Batting Style", "Bowling Style", "Playing Role"]
        list_var_store = []
        for i in range(len(list_names)):
            box=soup.find("p",text=list_names[i])
            if box ==None:
                list_var_store.append("")
            else:
                list_var_store.append(box.parent.span.get_text())


        Name = list_var_store[0]
        Born = list_var_store[1]
        Age = list_var_store[2]
        Batting_Style = list_var_store[3]
        Bowling_Style = list_var_store[4]
        Playing_Role = list_var_store[5]
        box = soup.find("p",text="TEAMS")
        if box ==None:
            National_Team = ""
        else:
            National_Team = box.parent.a.get_text()

        #career span
        elements = soup.find_all('span')
        if elements==None:
            career_span=""
        else:
            for element in elements:
                if 'INTL CAREER:' in element.text:
                    career_span = element.text

        pattern = r"\d{4}"
        years = re.findall(pattern, career_span)
        career_start = years[0]
        career_latest = years[1]

        # Player_image
        pattern = r"/([^/]+)-\d+$"
        matches = re.findall(pattern, url)
        name_url = matches[0]
        name_url = name_url.replace("-"," ").title().replace(" ","_")

        url2="https://en.wikipedia.org/wiki/"+name_url
        result = requests.get(url2)
        content = result.text
        soup = BeautifulSoup(content, "html")
        a = soup.find("span",class_="mw-default-size")
        if a==None:
            player_img=""
        else:
            a = a.find("img")
            player_img="https:"+a["src"]


        df1 =  pd.DataFrame([{"Name":Name,
                              "Born":Born,
                              "Age":Age,
                              "Batting_Style":Batting_Style,
                              "Bowling_Style":Bowling_Style,
                              "Playing_Role":Playing_Role,
                              "National_Team":National_Team,
                              "player_img":player_img,
                              "career_start":career_start,
                              "career_latest":career_latest}])
        #print(Name)
        df_player_profile = pd.concat([df_player_profile, df1], ignore_index=True)
        #print(ind)
        #print(ind)
        #bar()
    return df_player_profile

In [32]:
### National Teams & Flags

In [33]:
def scrape_National_flag(df_match):
    df_match=df_match
    team1 = list(df_match['Team_1'].unique())
    team2 = list(df_match['Team_2'].unique())
    total_teams = list(set(team2+team1))



    df_teams = pd.DataFrame(columns=["National_Team","national_flag"])

    #National_flag
    for i in total_teams:
        #print(i)
        National_Team = i.title()
        National_Team=National_Team.replace(" ","_")
        #print(National_Team)
        url3="https://en.wikipedia.org/wiki/Flag_of_"+National_Team
        result = requests.get(url3)
        content = result.text
        soup = BeautifulSoup(content, "html")
        a = soup.find_all("span",class_="mw-image-border")
        #print(a)
        if len(a)==0:
            national_flag=""
        else:
            national_flag = "https:"+a[0].find("img")["src"]
        #print(national_flag)
        df1 =  pd.DataFrame([{"National_Team":National_Team,
                              "national_flag":national_flag}])
        #print(Name)
        #print(df1.columns)
        df_teams = pd.concat([df_teams, df1], ignore_index=True) 
    
    return df_teams


### Player_Batting_stats

In [34]:
def scrape_batting_stats(df_players):
    with alive_bar(len(df_players),force_tty=True) as bar:

        df_batting_stats = pd.DataFrame(columns = ["Player_Name","matches","inns","not_out","runs_scored","HS",
                                     "batting_ave","ball_faced","batting_SR","100s","50s","fours","six"])
        df_players=df_players
        for ind in df_players.index:
            url ="https://www.espncricinfo.com/"+df_players["Player_link"][ind]
            #print(url)
            result = requests.get(url)
            content = result.text
            soup = BeautifulSoup(content, "lxml")

            Player_Name = df_players["Player_name"][ind]
            box2= soup.find("p",text="Batting & Fielding")
            #for row in box2.parent.tbody.find_all("tr")[:3]:
            for row in box2.parent.tbody.find_all("tr"):
                columns = row.find_all("td")
                if columns[0].span.get_text()=="ODI":
                    #print(columns[1].span.get_text())    


                    columns = row.find_all("td")
                    matches = columns[1].span.get_text()
                    inns = columns[2].span.get_text()
                    not_out = columns[3].span.get_text()
                    runs_scored = columns[4].span.get_text()
                    HS = columns[5].span.get_text()
                    batting_ave = columns[6].span.get_text()
                    ball_faced = columns[7].span.get_text()
                    batting_SR = columns[8].span.get_text()
                    hundreds = columns[9].span.get_text()
                    fiftys = columns[10].span.get_text()
                    fours = columns[11].span.get_text()
                    six = columns[12].span.get_text()

            df1 =  pd.DataFrame([{"Player_Name":Player_Name,
                        "matches":matches,
                        "inns":inns,
                        "not_out":not_out,
                        "runs_scored":runs_scored,
                        "ball_faced":ball_faced,
                        "HS":HS,
                        "batting_ave":batting_ave,
                        "batting_SR":batting_SR,
                        "100s":hundreds,
                        "50s":fiftys,
                        "fours":fours,
                        "six":six}])
                                  
                #print(Name)
            df_batting_stats = pd.concat([df_batting_stats, df1], ignore_index=True)
            bar()
    return df_batting_stats

### Player bowling stats

In [35]:
def scrape_bowling_stats(df_players):
    with alive_bar(len(df_players),force_tty=True) as bar:
        df_bowling_stats = pd.DataFrame(columns=["Player_Name","matches","inns","balls","runs","wkts","BBI",
                                    "bowling_ave","Econ","bowling_SR","5W","10W"])
        df_players=df_players
        for ind in df_players.index:
            url ="https://www.espncricinfo.com/"+df_players["Player_link"][ind]
            #print(url)
            result = requests.get(url)
            content = result.text
            soup = BeautifulSoup(content, "lxml")

            Player_Name = df_players["Player_name"][ind]
            box2= soup.find("p",text="Bowling")
            #for row in box2.parent.tbody.find_all("tr")[:3]:
            for row in box2.parent.tbody.find_all("tr"):
                columns = row.find_all("td")
                if columns[0].span.get_text()=="ODI":    
                    matches = columns[1].span.get_text()
                    inns = columns[2].span.get_text()
                    balls_bowled = columns[3].span.get_text()
                    runs_against = columns[4].span.get_text()
                    wkts = columns[5].span.get_text()
                    BBI = columns[6].span.get_text()
                    bowling_ave = columns[8].span.get_text()
                    Econ = columns[9].span.get_text()
                    bowling_SR = columns[10].span.get_text()
                    wickets_5 = columns[12].span.get_text()
                    wickets_10 = columns[13].span.get_text()

                    #print(T20_matches,T20_BBI,T20_Econ,T20_Econ,T20_5W)
            df1 =  pd.DataFrame([{"Player_Name":Player_Name,
                                  "matches":matches,
                                  "inns":inns,
                                  "balls_bowled":balls_bowled,
                                  "runs_against":runs_against,
                                  "wkts":wkts,
                                  "BBI":BBI,
                                  "bowling_ave":bowling_ave,
                                  "Econ":Econ,
                                  "bowling_SR":bowling_SR,
                                  "5W":wickets_5,
                                  "10W":wickets_10
                                  }])
            #print(Name)
            df_bowling_stats = pd.concat([df_bowling_stats, df1], ignore_index=True)
            #print(ind)
            bar()
    return df_bowling_stats

### Player Fielding Stats

In [36]:
def scrape_fielding_stats(df_players):
    with alive_bar(len(df_players),force_tty=True) as bar:
        df_fielding_stats = pd.DataFrame(columns = ["Player_Name","matches","catches","stumps"])
        
        df_players=df_players
        for ind in df_players.index:
            url ="https://www.espncricinfo.com/"+df_players["Player_link"][ind]
            #print(url)
            result = requests.get(url)
            content = result.text
            soup = BeautifulSoup(content, "lxml")

            Player_Name = df_players["Player_name"][ind]
            box2= soup.find("p",text="Batting & Fielding")
            for row in box2.parent.tbody.find_all("tr"):
                columns = row.find_all("td")
                if columns[0].span.get_text()=="ODI": 
                    matches = columns[1].span.get_text()
                    catches = columns[13].span.get_text()
                    stumps = columns[14].span.get_text()

            df1 =  pd.DataFrame([{"Player_Name":Player_Name,
                                  "matches":matches,
                                  "catches":catches,
                                  "stumps":stumps
                                  }])
            #print(Name)
            df_fielding_stats = pd.concat([df_fielding_stats, df1], ignore_index=True)
            bar()
    return df_fielding_stats

### scraping

In [42]:
url_list=["https://www.espncricinfo.com/records/year/team-match-results/2018-2018/one-day-internationals-2?decade=201",
         "https://www.espncricinfo.com/records/year/team-match-results/2019-2019/one-day-internationals-2?decade=201",
         "https://www.espncricinfo.com/records/year/team-match-results/2020-2020/one-day-internationals-2?decade=202",
         "https://www.espncricinfo.com/records/year/team-match-results/2021-2021/one-day-internationals-2?decade=202",
         "https://www.espncricinfo.com/records/year/team-match-results/2022-2022/one-day-internationals-2?decade=202",
         "https://www.espncricinfo.com/records/year/team-match-results/2023-2023/one-day-internationals-2?decade=202"]

for url in url_list:
    
    ######################################################################################
    print(url)
    
    catch = re.search(r"/(\d{4}-\d{4})/", url)
    if catch:
        result = catch.group(1)
        
    season = result
    ######################################################################################
    
    ######################################################################################
    csv_filename = "match_data.csv"
    csv_exists = os.path.isfile(csv_filename)
    mode = "w" if not csv_exists else "a"
    
    match = scrape_match_dataset(url)
    match["odi_year"]=season
    match.to_csv(csv_filename, mode=mode, index=False, header=not csv_exists)
    
    print("1")
    ######################################################################################
    
    ######################################################################################
    csv_filename = "match_links_data.csv"
    csv_exists = os.path.isfile(csv_filename)
    mode = "w" if not csv_exists else "a"
    
    match_links = scrape_match_links(url)
    match_links["odi_year"]=season
    match_links.to_csv(csv_filename, mode=mode, index=False, header=not csv_exists)
    
    print("2")
    ######################################################################################
    
    ######################################################################################
    csv_filename = "batting_scorecard_data.csv"
    csv_exists = os.path.isfile(csv_filename)
    mode = "w" if not csv_exists else "a"
    
    batting_scorecard = scrape_batting_summary(match_links,match)
    batting_scorecard["odi_year"]=season
    batting_scorecard.to_csv(csv_filename, mode=mode, index=False, header=not csv_exists)
    
    print("3")
    ######################################################################################
    
    ######################################################################################
    csv_filename = "bowling_scorecard_data.csv"
    csv_exists = os.path.isfile(csv_filename)
    mode = "w" if not csv_exists else "a"
    
    bowling_scorecard = scrape_bowling_summary(match_links,match)
    bowling_scorecard["odi_year"]=season
    bowling_scorecard.to_csv(csv_filename, mode=mode, index=False, header=not csv_exists)
    
    print("4")
    ######################################################################################
    
    ######################################################################################
    csv_filename = "player_links_data.csv"
    csv_exists = os.path.isfile(csv_filename)
    mode = "w" if not csv_exists else "a"
    
    player_links = scrape_player_links(match_links)
    player_links["odi_year"]=season
    player_links.to_csv(csv_filename, mode=mode, index=False, header=not csv_exists)
    
    print("5")
    ######################################################################################
    
    ######################################################################################
    csv_filename = "player_details_data.csv"
    csv_exists = os.path.isfile(csv_filename)
    mode = "w" if not csv_exists else "a"
    
    player_details = scrape_player_details(player_links)
    player_details["odi_year"]=season
    player_details.to_csv(csv_filename, mode=mode, index=False, header=not csv_exists)
    
    print("6")
    ######################################################################################
    
    ######################################################################################
    csv_filename = "National_flag_data.csv"
    csv_exists = os.path.isfile(csv_filename)
    mode = "w" if not csv_exists else "a"
    
    National_flag = scrape_National_flag(match)
    National_flag["odi_year"]=season
    National_flag.to_csv(csv_filename, mode=mode, index=False, header=not csv_exists)
        
    print("7")    
    ######################################################################################
    
    ######################################################################################
    csv_filename = "batting_stats_data.csv"
    csv_exists = os.path.isfile(csv_filename)
    mode = "w" if not csv_exists else "a"
    
    batting_stats = scrape_batting_stats(player_links)
    batting_stats["odi_year"]=season
    batting_stats.to_csv(csv_filename, mode=mode, index=False, header=not csv_exists)
        
    print("8")
    ######################################################################################
    
    ######################################################################################
    csv_filename = "bowling_stats_data.csv"
    csv_exists = os.path.isfile(csv_filename)
    mode = "w" if not csv_exists else "a"
    
    bowling_stats = scrape_bowling_stats(player_links)
    bowling_stats["odi_year"]=season
    bowling_stats.to_csv(csv_filename, mode=mode, index=False, header=not csv_exists)
    
    print("9")
    ######################################################################################
    
    ######################################################################################
    csv_filename = "fielding_stats_data.csv"
    csv_exists = os.path.isfile(csv_filename)
    mode = "w" if not csv_exists else "a"
    
    fielding_stats = scrape_fielding_stats(player_links)
    fielding_stats["odi_year"]=season
    fielding_stats.to_csv(csv_filename, mode=mode, index=False, header=not csv_exists)
    
    print("10")
    ######################################################################################

https://www.espncricinfo.com/records/year/team-match-results/2018-2018/one-day-internationals-2?decade=201
1
2
3
4
5
6
7
|████████████████████████████████████████| 362/362 [100%] in 2:28.9 (2.42/s)    
8
|████████████████████████████████████████| 362/362 [100%] in 1:53.4 (3.19/s)    
9
|████████████████████████████████████████| 362/362 [100%] in 1:08.4 (5.30/s)    
10
https://www.espncricinfo.com/records/year/team-match-results/2019-2019/one-day-internationals-2?decade=201
1
2
3
4
5
6
7
|████████████████████████████████████████| 402/402 [100%] in 2:57.1 (2.27/s)    
8
|████████████████████████████████████████| 402/402 [100%] in 2:28.3 (2.71/s)    
9
|████████████████████████████████████████| 402/402 [100%] in 3:22.6 (1.98/s)    
10
https://www.espncricinfo.com/records/year/team-match-results/2020-2020/one-day-internationals-2?decade=202
1
2
3
4
5
6
7
|████████████████████████████████████████| 253/253 [100%] in 1:20.8 (3.12/s)    
8
|████████████████████████████████████████| 253/253 [10

In [135]:
a= pd.read_csv("fielding_stats_wc.csv")
a

,Unnamed: 0,Player_Name,matches,catches,stumps,WC_year
0,0,Martin Guptill,198,104,0,2014to15-2014to15
1,1,Brendon McCullum,260,262,15,2014to15-2014to15
2,2,Kane Williamson,161,64,0,2014to15-2014to15
3,3,Ross Taylor,236,142,0,2014to15-2014to15
4,4,Grant Elliott,83,17,0,2014to15-2014to15
...,...,...,...,...,...,...
200,200,James Tredwell,45,14,0,2014to15-2014to15
201,201,Johnson Charles,58,25,2,2014to15-2014to15
202,202,Ehsan Adil,6,0,0,2014to15-2014to15
203,203,Tharindu Kaushal,1,0,0,2014to15-2014to15


In [23]:
import os

In [24]:
url_list=["https://www.espncricinfo.com/records/year/team-match-results/2018-2018/one-day-internationals-2?decade=201",
         "https://www.espncricinfo.com/records/year/team-match-results/2019-2019/one-day-internationals-2?decade=201",
         "https://www.espncricinfo.com/records/year/team-match-results/2020-2020/one-day-internationals-2?decade=202",
         "https://www.espncricinfo.com/records/year/team-match-results/2021-2021/one-day-internationals-2?decade=202",
         "https://www.espncricinfo.com/records/year/team-match-results/2022-2022/one-day-internationals-2?decade=202",
         "https://www.espncricinfo.com/records/year/team-match-results/2023-2023/one-day-internationals-2?decade=202"]

for url in url_list:
    
    print(url)
    
    csv_filename = "match_data.csv"
    csv_exists = os.path.isfile(csv_filename)
    # Create or append to the CSV file based on existence
    mode = "w" if not csv_exists else "a"
    
    catch = re.search(r"/(\d{4}-\d{4})/", url)
    if catch:
        result = catch.group(1)
        
    season = result
    
    match = scrape_match_dataset(url)
    print(len(match))
    match["odi_year"]=season
    match.to_csv(csv_filename, mode=mode, index=False, header=not csv_exists)

https://www.espncricinfo.com/records/year/team-match-results/2018-2018/one-day-internationals-2?decade=201
128
https://www.espncricinfo.com/records/year/team-match-results/2019-2019/one-day-internationals-2?decade=201
150
https://www.espncricinfo.com/records/year/team-match-results/2020-2020/one-day-internationals-2?decade=202
44
https://www.espncricinfo.com/records/year/team-match-results/2021-2021/one-day-internationals-2?decade=202
71
https://www.espncricinfo.com/records/year/team-match-results/2022-2022/one-day-internationals-2?decade=202
161
https://www.espncricinfo.com/records/year/team-match-results/2023-2023/one-day-internationals-2?decade=202
122


In [43]:
a=pd.read_csv("match_data.csv")

In [44]:
len(a)

1608